In [3]:
!pip install tf_agents

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tf_agents-0.15.0-py3-none-any.whl (1.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.0 MB/s eta 0:00:00
  Using cached gym-0.23.0.tar.gz (624 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697662 sha256=766170291517d75d630f47b670b7263229bbf01205b0c3cf6051bd86a6a13619
  Stored in directory: /root/.cache/pip/wheels/96/b9/bb/994c1324b65e39dd1cd7b8ba92e5fb766dd77980929414a866
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [6]:
pip install movielens_py_environment


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement movielens_py_environment (from versions: none)
ERROR: No matching distribution found for movielens_py_environment


In [7]:
import tensorflow as tf
from tf_agents.bandits.agents import lin_ucb_agent, neural_epsilon_greedy_agent
from tf_agents.bandits.agents import linear_thompson_sampling_agent
from tf_agents.bandits.metrics import tf_metrics
import matplotlib.pyplot as plt
import numpy as np
from movielens_py_environment import MovieLensPyEnvironment
from tf_metrics import RegretMetric
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network

ModuleNotFoundError: ignored

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
env = MovieLensPyEnvironment(data_dir='/content/gdrive/MyDrive/ml-100k', rank_k=20)

In [ ]:
tf_env = tf_py_environment.TFPyEnvironment(env)

In [ ]:
regret_metric = tf_metrics.RegretMetric(tf_env.action_spec().maximum + 1)

In [ ]:
num_episodes = 200
num_iterations = 100

In [ ]:
q_net = q_network.QNetwork(
    input_tensor_spec=tf_env.time_step_spec().observation,
    action_spec=tf_env.action_spec(),
    fc_layer_params=(50,)
)

In [ ]:
agents = [
    lin_ucb_agent.LinearUCBAgent(time_step_spec=tf_env.time_step_spec(),
                                  action_spec=tf_env.action_spec()),
    linear_thompson_sampling_agent.LinearThompsonSamplingAgent(time_step_spec=tf_env.time_step_spec(),
                                                     action_spec=tf_env.action_spec()),
    neural_epsilon_greedy_agent.NeuralEpsilonGreedyAgent(
        time_step_spec=tf_env.time_step_spec(),
        action_spec=tf_env.action_spec(),
        reward_network=q_net,
        optimizer=tf.compat.v1.train.AdamOptimizer(),
        epsilon=0.1,
    ),
]

In [ ]:
for agent in agents:
    # Reset the environment
    time_step = tf_env.reset()

    # Initialize the agent
    agent.initialize()

    # Initialize the regret metric
    regret_metric.reset()

    # Initialize the arrays to store the regret and rewards
    regret = np.zeros(num_iterations)
    rewards = np.zeros(num_iterations)

    for i in range(num_iterations):
        # Collect a step from the environment
        action_step = agent.collect_policy.action(time_step)
        next_time_step = tf_env.step(action_step.action)

        # Update the agent
        agent.train(
            experience=(time_step, action_step, next_time_step))

        # Update the regret and reward
        regret_metric.update(time_step, action_step, next_time_step)
        regret[i] = regret_metric.result().numpy()

        rewards[i] = next_time_step.reward.numpy()

        # Update the time step
        time_step = next_time_step

    

ValueError: ignored